## Описание проекта:
Работаем в стартапе, который продаёт продукты питания. Нужно разобраться, как ведут себя пользователи нашего мобильного приложения. 
1. Изучим воронку продаж. Узнаем, как пользователи доходят до покупки. Сколько пользователей доходит до покупки, а сколько — «застревает» на предыдущих шагах? На каких именно?
2. После этого исследуем результаты A/A/B-эксперимента. Дизайнеры захотели поменять шрифты во всём приложении, а менеджеры испугались, что пользователям будет непривычно. Договорились принять решение по результатам A/A/B-теста. Пользователей разбили на 3 группы: 2 контрольные со старыми шрифтами и одну экспериментальную — с новыми. Выясним, какой шрифт лучше.
3. Создание двух групп A вместо одной имеет определённые преимущества. Если две контрольные группы окажутся равны, мы можем быть уверены в точности проведенного тестирования. Если же между значениями A и A будут существенные различия, это поможет обнаружить факторы, которые привели к искажению результатов. Сравнение контрольных групп также поможет понять, сколько времени и данных потребуется для дальнейших тестов.

В случае общей аналитики и A/A/B-эксперимента работайте с одними и теми же данными. В реальных проектах всегда идут эксперименты. Аналитики исследуют качество работы приложения по общим данным, не учитывая принадлежность пользователей к экспериментам.

#### Описание данных:
* Каждая запись в логе — это действие пользователя, или событие. 
* EventName — название события;
* DeviceIDHash — уникальный идентификатор пользователя;
* EventTimestamp — время события;
* ExpId — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.


## 1. Посмотрим на данные и импортируем нужные библиотеки

In [1]:
import re
import pandas as pd
import numpy as np
import scipy.stats as stats
import plotly.express as px # пробуем plotly
import plotly.graph_objects as go
import plotly.subplots as sp
import math
from datetime import datetime, timedelta
from plotly.subplots import make_subplots
from scipy import stats as st

In [2]:
try:
    logs = pd.read_csv('/datasets/logs_exp.csv', sep='\t')
except:
    logs = pd.read_csv('https://code.s3.yandex.net//datasets/logs_exp.csv', sep='\t')


In [3]:
logs.head()

,EventName,DeviceIDHash,EventTimestamp,ExpId
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248


In [4]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   EventName       244126 non-null  object
 1   DeviceIDHash    244126 non-null  int64 
 2   EventTimestamp  244126 non-null  int64 
 3   ExpId           244126 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 7.5+ MB


Пропусков нет, названия стоблцов не в Python формате, поле EventTimestamp имеет тип int64, а это время.

## 2. Подготовим данные

In [5]:
logs.duplicated().sum() # проверим на дублиткаты

413

In [6]:
logs = logs.drop_duplicates().reset_index(drop=True) # удаляю явные дубликаты

In [7]:
logs.columns = (logs.columns.str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True).str.lower()) # столбцы в snake_case
logs['date_time'] = pd.to_datetime(logs['event_timestamp'] , unit='s') # добавляем стобец с датой и временем
logs['date'] = pd.to_datetime(logs['date_time']).dt.date # добавляем столбец с датой
logs['date'] = pd.to_datetime(logs['date']) # приводим к типу datetime
logs.head()

,event_name,device_idhash,event_timestamp,exp_id,date_time,date
0,MainScreenAppear,4575588528974610257,1564029816,246,2019-07-25 04:43:36,2019-07-25
1,MainScreenAppear,7416695313311560658,1564053102,246,2019-07-25 11:11:42,2019-07-25
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248,2019-07-25 11:28:47,2019-07-25
3,CartScreenAppear,3518123091307005509,1564054127,248,2019-07-25 11:28:47,2019-07-25
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248,2019-07-25 11:48:42,2019-07-25


Привёл название столбцов к snake_case, проверил данные на пропуски, добавил столбцы и привел их к формату datetime и удалил явные дубликаты.

## 3. Изучим и проверим данные

In [8]:
logs.nunique() # уникальные значения по столбцам

event_name              5
device_idhash        7551
event_timestamp    176654
exp_id                  3
date_time          176654
date                   14
dtype: int64

* 5 названий события.
* 7551 уникальных индинтификаторов пользователй
* 3 номера групп эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.
* 14 дат в которых проводились наблюдения

In [9]:
print(f"Всего событий в логе:{logs['event_name'].count()}")

Всего событий в логе:243713


In [10]:
print(f"Среднее количество событий на каждого пользователя: {round(logs['event_name'].count() / logs['device_idhash'].nunique())}")

Среднее количество событий на каждого пользователя: 32


14 дней в которые проводились эксперементы.
Найдем начальную дату и конечную

In [11]:
print(f"Минимальная дата наблюдений: {logs['date_time'].min()}")
print(f"Максимальная дата наблюдений: {logs['date_time'].max()}")

Минимальная дата наблюдений: 2019-07-25 04:43:36
Максимальная дата наблюдений: 2019-08-07 21:15:17


По графику видно что у вас не одинаково полные данные за весь период, с 25.07.2019 и до 01.08.2019 данные не полные.

In [12]:

new_logs = logs.query("date > '2019-07-31'") # Убираю неполные данные
new_logs.nunique()


event_name              5
device_idhash        7534
event_timestamp    174044
exp_id                  3
date_time          174044
date                    7
dtype: int64

In [13]:
#количество событий
print(
    '\n Количество событий после чистки данных:', new_logs.shape[0],
    '\n Количество потеряных событий:', logs.shape[0] - new_logs.shape[0],
    '\n % событий от первоначального:', round(new_logs.shape[0]/logs.shape[0]*100, 2)
     )

#количество уникальных пользователей
print(
    '\n Количество пользователей после чистки данных:', len(new_logs['device_idhash'].unique()),
    '\n Количество потеряных пользователей:', len(logs['device_idhash'].unique()) - len(new_logs['device_idhash'].unique()),
    '\n % пользователей от первоначального:', round(new_logs['device_idhash'].nunique() / logs['device_idhash'].nunique()*100, 2))


 Количество событий после чистки данных: 240887 
 Количество потеряных событий: 2826 
 % событий от первоначального: 98.84

 Количество пользователей после чистки данных: 7534 
 Количество потеряных пользователей: 17 
 % пользователей от первоначального: 99.77


* Потеряли 2826 наблюдений что составляет 1.16%.
* Количество "потерянных" пользователей 17 что составляет 0.23%

Потери как наблюдений так и пользователей не существенные.

### Вывод:
1. Всего событий в логе: 243713
2. 
  * 5 названий события.
  * 7551 уникальных индинтификаторов пользователй
  * 3 номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.
  * 14 дат в которых проводились наблюдения
3. Среднее количество событий на каждого пользователя: 32
4. 
  * Минимальная дата наблюдений: 2019-07-25 04:43:36
  * Максимальная дата наблюдений: 2019-08-07 21:15:17
5. Отбросив не полные данные потеряли 17 пользователей и 2610 наблюдений. 

## 4. Изучим воронку событий

In [14]:
logs['event_name'].unique() # Смотрим какие события есть в логах

array(['MainScreenAppear', 'PaymentScreenSuccessful', 'CartScreenAppear',
       'OffersScreenAppear', 'Tutorial'], dtype=object)

1. MainScreenAppear - появление главного экрана
2. OffersScreenAppear - появление экрана с оффертой;
3. CartScreenAppear - появление экрана с корзиной;
4. PaymentScreenSuccessful - экран об успешной оплате;
5. Tutorial - ознакомление с инструкцией.

In [15]:
new_logs['event_name'].value_counts() # посмотрим на распределением логов между событиями

MainScreenAppear           117328
OffersScreenAppear          46333
CartScreenAppear            42303
PaymentScreenSuccessful     33918
Tutorial                     1005
Name: event_name, dtype: int64

In [16]:
# рассчет количества пользователей по всем возможным событиям.
event_user = (new_logs
                .groupby('event_name')
                .agg({'device_idhash':'nunique'})
                .reset_index()
                .sort_values(by='device_idhash', ascending=False)
                .reset_index(drop=True)
                )
event_user.columns = ['event_name', 'user_cnt']
event_user

,event_name,user_cnt
0,MainScreenAppear,7419
1,OffersScreenAppear,4593
2,CartScreenAppear,3734
3,PaymentScreenSuccessful,3539
4,Tutorial,840


Места по количеству пользователей для каждого события распределились точно так же, как и по частоте событий.

Создаем таблицу для анализа

In [17]:
# Группируем данные по событию, его частоте и количеству уникальных пользователей
event_user = (
        new_logs.groupby('event_name').agg({'event_name':'count', 'device_idhash': 'nunique'})
        .rename(columns={'event_name':'event_cnt', 'device_idhash':'user_cnt'})
        .sort_values(by ='event_cnt', ascending=False).reset_index()
    )

# Доля уникальных пользователей относительно общего количества пользователей
event_user['rate %'] = round((event_user['user_cnt'] / len(new_logs['device_idhash'].unique())) * 100, 1)

#  Cдвиг столбца user_cnt на одну строку вниз
event_user['step'] = event_user['user_cnt'].shift()

# Конверсия в следущий шаг
event_user['convers_step'] = round(event_user['user_cnt'] / event_user['step'] * 100, 1) 

# Удаляем столбец step
event_user.drop(columns= ['step'], axis = 1, inplace = True)

# Заменю NaN в первой строке на 100% к ним будем считать следующий шаг
event_user = event_user.fillna(100)

event_user


,event_name,event_cnt,user_cnt,rate %,convers_step
0,MainScreenAppear,117328,7419,98.5,100.0
1,OffersScreenAppear,46333,4593,61.0,61.9
2,CartScreenAppear,42303,3734,49.6,81.3
3,PaymentScreenSuccessful,33918,3539,47.0,94.8
4,Tutorial,1005,840,11.1,23.7


Видно что не все пользователи заходят на главный экран, возможно, они уходят на этапе туториала. Возможны проблемы с юзабельность инструкции их сложностью для понимания. Или часть приходит минуя главный экран.

Количества пользователей для каждого события распределились:
1. На главный экран зашло 7 419 пользователей. 
2. Экран с оффертой открыли 4 593 пользователя. 
3. Экран с корзиной появился у 3 734 пользователей.
4. Экран с успешной оплатой - у 3 539 пользователей. 
5. Инструкцию открывало 840 пользователей, что указывает на то, что не каждому пользователю для совершения всех шагов до оплаты нужен туториал.

Частота событий:
1. Самое частое событие - открытие главного экрана (117 328 раз).
2. Экран с предложениями (46 333 раза). 
3. Корзина (42 303 раза).
4. Успешная оплата (33 918 раза). 
5. Туториал (1 005 раз). 

Возможно, пользователи не считают необходимым ознакомиться с инструкцией и обращаются к ней только в случае необходимости.
Поэтому учитывать Tutorial в воронке наверняка не нужно.

In [18]:
event_user = event_user.query('event_name != "Tutorial"').reset_index(drop=True) # Убираем Tutorial 

Из проведенного анализа воронки событий следует, что переход на экран офферты является наиболее сложным для пользователей этапом. Его успешно проходит только 62% пользователей, которые заходят на главный экран Затем на следующем шаге 82 пользователей, которые добрались до экрана с оффертой, переходят в корзину. И, наконец, из тех, кто попадает в корзину, 95% завершают покупку. Рискну предположить что в приложении  возможны проблемы с юзабилити на главном экране, которые приводят к оттоку пользователей.

In [19]:
result = (
    event_user.loc[event_user['event_name']=='PaymentScreenSuccessful', 'user_cnt'].sum() / 
    event_user.loc[event_user['event_name']=='MainScreenAppear', 'user_cnt'].sum() * 100
)
print("Процент пользователей, прошедших от первого события до оплаты: {:.2f}%".format(result))

Процент пользователей, прошедших от первого события до оплаты: 47.70%


### Вывод:
 * Не все пользователи заходят на главный экран, возможно, они уходят на этапе туториала.И инструкции в приложении возможно имеют проблемы(сложность, юзабельность и т.д)
 * Места по количеству пользователей и по частоте событий распределились одинаково.
 1. Самое частое событие - открытие главного экрана.
 2. Экран с предложениями.
 3. Корзина.
 4. Успешная оплата.
 5. Туториал 
 
 * Выяснили что что переход на экран офферты является наиболее сложным для пользователей этапом.
 * Предположили что в приложении возможны проблемы с юзабилити на главном экране, которые приводят к оттоку пользователей.
 * Посчитали процент пользователей, прошедших от первого события до оплаты: 47.70% 

## 5. Изучим результат эксперемента

In [20]:
new_logs['exp_id'].value_counts() # распределение событий по группам

248    84563
246    79302
247    77022
Name: exp_id, dtype: int64

Больше всего событий в эксперементальной группе 248, второе место у группы 246 и на последнем месте 247.
* Kоличество событий в эксперементальной 248 группе на 6.74% больше чем 246 и 9.79% выше  чем в 247.

In [21]:
new_logs.groupby('exp_id')['device_idhash'].nunique().sort_values(ascending=False) # количество пользователей по группам

exp_id
248    2537
247    2513
246    2484
Name: device_idhash, dtype: int64

Группы сформированы ровно без большой разницы по количеству участников.
Но больше всех пользователей в эксперементальной группе.

In [22]:
user_group_counts = new_logs.groupby('device_idhash')['exp_id'].nunique()
users_in_group = user_group_counts[user_group_counts > 1].index.tolist()
print(f"Количество пользователей входящих более чем в одну группу: {len(users_in_group)}")

Количество пользователей входящих более чем в одну группу: 0


In [23]:
def event_group_pivot(group):
    result = (
        new_logs
        .query('exp_id == @group and event_name != "Tutorial"')
        .groupby('event_name')
        .agg(device_count=('device_idhash', 'nunique'))
        .sort_values(by='device_count', ascending=False)
        .reset_index()
    )
    return result

def event_group_ratio(df):
    unique_devices = new_logs['device_idhash'].nunique()
    df['ratio'] = round((df['device_count'] / unique_devices), 3)

# Таблицы для каждой группы
event_246_pivot = event_group_pivot(246)
event_group_ratio(event_246_pivot)
event_246_pivot.columns = ['event_name', '246', '246_pr_user']

event_247_pivot = event_group_pivot(247)
event_group_ratio(event_247_pivot)
event_247_pivot.columns = ['event_name', '247', '247_pr_user']

event_248_pivot = event_group_pivot(248)
event_group_ratio(event_248_pivot)
event_248_pivot.columns = ['event_name', '248', '248_pr_user']

# Объединение таблиц
event_group_pivot = (
    event_246_pivot
    .merge(event_247_pivot, on='event_name')
    .merge(event_248_pivot, on='event_name')
    )

event_group_pivot

,event_name,246,246_pr_user,247,247_pr_user,248,248_pr_user
0,MainScreenAppear,2450,0.325,2476,0.329,2493,0.331
1,OffersScreenAppear,1542,0.205,1520,0.202,1531,0.203
2,CartScreenAppear,1266,0.168,1238,0.164,1230,0.163
3,PaymentScreenSuccessful,1200,0.159,1158,0.154,1181,0.157


На первый взгляд группа 246 кажется немного более успешной, чем остальные группы на почти всех этапах. Группы 247 и 248 практически не различаются друг от друга (за исключением этапа с корзиной).

Данные за сутки имеют нормальное распределение с небольшими отклонениями в отдельные дни. Это позволяет сформулировать и проверить гипотезу о равенстве долей по событиям для разных групп.Можно использовать Z-критерий для разности долей. Этот критерий основан на нормальном распределении разности выборочных долей и позволяет проверять гипотезу о равенстве долей между двумя группами.

### Исследование разницы между выборками A/A-теста

Определим гипотезы:
* Гипотеза H0 доли двух выборок уникальных посетителей, побывавших на этапе воронки, равны между собой.
* Гипотеза H1 между долями уникальных посетителей, побывавших на этапе воронки, есть значимая разница..
* Уровень значимости выберем равным 1%, что означает, что вероятность ошибочно отвергнуть H0 при ее истинности не должна превышать 1%.
Понизил уровень значимости потомучто тестировать будем 4 группы между собой,  учитывая тот факт, что у нас идет множественная проверка гипотез, а значит увеличивается риск ложноположительного результата, найти различия там, где их на самом деле нет.

Функция для проведения z-теста для проверки гипотезы о равенстве долей двух выборок

In [24]:
def z_test(exp_group_1, exp_group_2, event, alpha):
    
    alpha = alpha
    
    group_1 = new_logs.query('exp_id == @exp_group_1')
    group_2 = new_logs.query('exp_id == @exp_group_2')

    #значения выборок на уровне тестируемого события
    successes = np.array([event_group_pivot.query('event_name == @event')[str(exp_group_1)].sum(),
                          event_group_pivot.query('event_name == @event')[str(exp_group_2)].sum()])
    
    #первоначальные значения выборок
    trials = np.array([len(group_1['device_idhash'].unique()), len(group_2['device_idhash'].unique())])
    
    # пропорция успехов в группах:
    p1 = successes[0]/trials[0]
    p2 = successes[1]/trials[1]
    
    # пропорция успехов в комбинированном датасете:
    p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])
    
    # разница пропорций в датасетах
    difference = p1 - p2
    
    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference / math.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1]))
    
    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1)
    
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    
    print('p-значение: ', round(p_value, 4))
    if (p_value < alpha):
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.')


In [25]:
z_test(246, 247, 'MainScreenAppear', 0.01)

p-значение:  0.7571
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.


In [26]:
z_test(246, 247, 'OffersScreenAppear', 0.01)

p-значение:  0.2481
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.


In [27]:
z_test(246, 247, 'CartScreenAppear', 0.01)

p-значение:  0.2288
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.


In [28]:
z_test(246, 247, 'PaymentScreenSuccessful', 0.01)

p-значение:  0.1146
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.


#### Вывод:
Исходя из результата всех A/A-тестов, где было проверено отсутствие статистически значимых различий в долях пользователей, совершивших каждое из событий, можно заключить, что разделение на группы происходит корректно.

### Иследование разницы между выборками А/В-теста

In [29]:
event_group_pivot['Cluster_246_247'] = (event_group_pivot['246'] + event_group_pivot['247'])
event_group_pivot['Cluster_246_247'] = event_group_pivot['Cluster_246_247'].astype('int')
event_group_pivot

,event_name,246,246_pr_user,247,247_pr_user,248,248_pr_user,Cluster_246_247
0,MainScreenAppear,2450,0.325,2476,0.329,2493,0.331,4926
1,OffersScreenAppear,1542,0.205,1520,0.202,1531,0.203,3062
2,CartScreenAppear,1266,0.168,1238,0.164,1230,0.163,2504
3,PaymentScreenSuccessful,1200,0.159,1158,0.154,1181,0.157,2358


Функция для сравнения с объединённой группой.

In [30]:
def z_test_united(exp_group_2, event, alpha):
    
    alpha=alpha
    group_1 = new_logs.query('exp_id == 246 | exp_id == 247')
    group_2 = new_logs.query('exp_id == @exp_group_2')

    successes = np.array([event_group_pivot.query('event_name == @event')['Cluster_246_247'].sum(),
                          event_group_pivot.query('event_name == @event')[str(exp_group_2)].sum()])
    trials = np.array([len(group_1['device_idhash'].unique()), len(group_2['device_idhash'].unique())])
    
    p1 = successes[0]/trials[0]
    p2 = successes[1]/trials[1]
    p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])
    
    difference = p1 - p2
    
    z_value = difference / math.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1]))
    distr = st.norm(0, 1)
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    
    print('p-значение: ', round(p_value, 4))
    if (p_value < alpha):
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.')

#### A/B-тест для события "MainScreenAppear"


In [31]:
z_test(246, 248, 'MainScreenAppear', 0.01)
z_test(247, 248, 'MainScreenAppear', 0.01)
z_test_united(248, 'MainScreenAppear', 0.01)

p-значение:  0.295
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.
p-значение:  0.4587
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.
p-значение:  0.2942
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.


##### Нет оснований считать доли на первом шаге разными.

#### A/B-тест для события "OffersScreenAppear"

In [32]:
z_test(246, 248, 'OffersScreenAppear', 0.01)
z_test(247, 248, 'OffersScreenAppear', 0.01)
z_test_united(248, 'OffersScreenAppear', 0.01)

p-значение:  0.2084
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.


p-значение:  0.9198
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.
p-значение:  0.4343
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.


##### Нет оснований считать доли на втором шаге разными.

#### A/B-тест для события 'CartScreenAppear'

In [33]:
z_test(246, 248, 'CartScreenAppear', 0.01)
z_test(247, 248, 'CartScreenAppear', 0.01)
z_test_united(248, 'CartScreenAppear', 0.01)

p-значение:  0.0784
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.
p-значение:  0.5786
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.
p-значение:  0.1818
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.


##### Нет оснований считать доли на третьем шаге разными.

#### A/B-тест для события 'PaymentScreenSuccessful'

In [34]:
z_test(246, 248, 'PaymentScreenSuccessful', 0.01)
z_test(247, 248, 'PaymentScreenSuccessful', 0.01)
z_test_united(248, 'PaymentScreenSuccessful', 0.01)

p-значение:  0.2123
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.
p-значение:  0.7373
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.
p-значение:  0.6004
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.


##### И на четвёртом шаге существенных различий в долях между группами не обнаружено.

##### Вывод:
-  4 z-теста контрольных групп 246 и 247 в разрезе событий воронки :  статистически достоверных отличий между группами не выявлено, что означает, что выборки сделаны корректно.

-  12 z-теста аналогичных тестов с контрольными и тестируемой группой:  статистически достоверных отличий между конверсиями групп не выявлено.

Поскольку в итоге было проведено 16 тестов, то для того, чтобы групповая вероятность ошибки не превышала определенный уровень значимости α (мы понизили уровень значимости до 0.01), если бы количество групп было больше нами была бы применена поправка Бонферрони к коэффициенту статистической значимости.


На основе проведенных A/B-тестов мы можем сделать вывод о том, что изменения в дизайне сайта(изменение шрифтов) не привели к статистически значимым изменениям в поведении пользователей. В целом результаты тестов показывают, что пользователи быстро адаптировались к новому дизайну и продолжили свое обычное поведение. Мы можем сделать вывод о том, что изменения в дизайне не оказали значительного влияния на пользователей.


## Основной вывод:
 Из представленных данных состоит в том, что приложение имеет проблемы с юзабилити на экране офферты, что приводит к оттоку пользователей. Более того, изменение шрифтов, не привели к статистически значимым изменениям в поведении пользователей. Также, можно заключить, что разделение на группы в рамках проведенных A/A-тестов происходит корректно. В целом, проведенный эксперемент позволяет сделать выводы о текущем состоянии приложения и о том, какие меры могут быть приняты для улучшения его юзабилити.
 Кроме того, стоит учесть, что выводы, сделанные на основе A/B-тестирования, зависят от выбранного уровня значимости. Например, если бы мы выбрали уровень значимости 10%, мы бы обнаружили значительные различия между группами 246 и 248 в отношении появления корзины. Однако при таком уровне значимости каждый десятый раз можно получить ложный результат, а мы провели 16 тестов. Поэтому при выборе уровня значимости следует быть внимательным и учитывать его влияние на результаты и выводы.

<div class="alert alert-block alert-info">

### Комментарий студента:
Спасибо Вам за все то что вы делаете. Всем Добра Удачи и Хорошего на
</div>